In [ ]:
from datascience import *
import numpy as np
## Normal Distribution
%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')
from IPython.display import Image
from IPython.core.display import HTML 

In [ ]:
voters = Table().with_columns("Candidate", ["A", "B"], "Chance", [0.6, 0.4])

In [ ]:
def r_scatter(r):
    plots.figure(figsize=(5,5))
    "Generate a scatter plot with a correlation approximately r"
    x = np.random.normal(0, 1, 1000)
    z = np.random.normal(0, 1, 1000)
    y = r*x + (np.sqrt(1-r**2))*z
    plots.scatter(x, y)
    plots.xlim(-4, 4)
    plots.ylim(-4, 4)

## Part 1. Confidence Intervals and Sample Size


In [ ]:
#http://inferentialthinking.com/notebooks/san_francisco_2015.csv
sf = Table.read_table('san_francisco_2015.csv').select(3, 11, 21)
sf.set_format(2, NumberFormatter(0))
sf = sf.where(2, are.above(10000))
sf.show(3)

In [ ]:
comp_bins = np.arange(0, 700000, 25000)
sf.hist(2, bins=comp_bins, unit='dollar')

In [ ]:
# confidence interval methods

def bootstrap_mean(sample_from_population, label, repetitions):
    resampled_means = []
    for i in np.arange(repetitions):
        resample = sample_from_population.sample()
        mean = np.mean(resample.column(label))
        resampled_means.append(mean)
    return resampled_means

def bootstrap_ci_mean(sample_from_population, label, repetitions):
    resampled_means = bootstrap_mean(sample_from_population, label, repetitions)
    
    interval_95 = make_array(
        percentile(2.5, resampled_means),
        percentile(97.5, resampled_means)
    )
    
    Table().with_column('Resampled mean', resampled_means).hist(0)
    plots.plot(interval_95, [0, 0], color='gold', lw=8)
    print('Approximate 95% Bootstrap Confidence Interval for Population Mean:')
    print(np.round(interval_95, 3))
    print('Interval Width: '+str(interval_95[1] - interval_95[0]))

In [ ]:
sf_sample = sf.sample(200)
sf_sample.show(3)

In [ ]:
#resample 1000 times from my original sample (with replacement)
#display histogram showing 95% confidence interval

bootstrap_ci_mean(sf_sample, 'Total Compensation', 1000)

In [ ]:
# Sample size was 200. Confidence interval is too wide
# Note, numbers will change because samples are random! 

In [ ]:
# What do you think the sample size should be to achieve a 
# 95% confidence interval with width of $10,000 or less?

# A: 250
# B: 300
# C: 400
# D: 800
# E: 10,000

In [ ]:
sf_sample = sf.sample(800)
bootstrap_ci_mean(sf_sample, 'Total Compensation', 1000)

In [ ]:
#(back to slides to recap)

## Experiment design


In [ ]:
#potential population

votes = [1, 1, 1, 1, 1, 1, 1, 1, 1, 0]
np.std(votes)

In [ ]:
votes = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
np.std(votes)

In [ ]:
votes = [1, 1, 1, 1, 1, 1, 0, 0, 0, 0]
np.std(votes)

In [ ]:
total_voters=10

def sd_voters(n_voters_for_a):
    votes = np.append(np.ones(n_voters_for_a), np.zeros(total_voters - n_voters_for_a))
    print(votes)
    return np.std(votes)

sd_voters(6)

In [ ]:
# Fact: if data contains only 0-1 then SD will never be above 0.5
# we know that worst_sd_pop = 0.5
# width_in_sds = 4
# desired_width = 0.03

In [ ]:
# √(sample size)  ≥  4 x (SD of 0-1 population) / 0.03
worst_sd_pop = 0.5
width_in_sds = 4
desired_width = 0.03
(width_in_sds * (worst_sd_pop / desired_width)) ** 2

In [ ]:
#  no matter what my population looks like, 
# if I take a sample of 4445 people, my CI will always be 0.03 width or less

## Conducting the experiment


In [ ]:
voters.labels

In [ ]:
voters.select(0)

In [ ]:
# weights - Array specifying probability. Must be a valid probability distribution 

observed_sample = voters.select(0).sample(1000, weights=voters.column('Chance'))
observed_sample.show(3)

In [ ]:
# means are proportions

n = 4445
observed_sample = voters.select(0).sample(n, weights=voters.column('Chance'))

means = []
for i in np.arange(1000):
    resample = observed_sample.sample()
    means.append(np.count_nonzero(resample.column(0) == 'A') / n) 
    
print(percentile(2.5, means), percentile(97.5, means))

In [ ]:
0.6161979752530934 - 0.5808773903262092

In [ ]:
voters

## Part 2. Prediction

In [ ]:
# Reminder:

#http://inferentialthinking.com/notebooks/galton.csv
galton = Table.read_table('galton.csv')



heights = Table().with_columns(
    'MidParent', galton.column('midparentHeight'),
    'Child', galton.column('childHeight')
    )

heights.scatter(0)


In [ ]:
#prediction

def predict_child(parent):
    """Return a prediction of the height of a child 
    whose parents have a midparent height of parent.
    
    The prediction is the average height of the children 
    whose midparent height is in the range mpht plus or minus 0.5 inches.
    """
    close_points = heights.where('MidParent', are.between(parent - 0.5, parent + 0.5))
    return close_points.column('Child').mean()

heights_and_predict = heights.with_column(
    'Prediction', heights.apply(predict_child, 'MidParent')
)

heights_and_predict.scatter(0)

In [ ]:
# Goal: perfect straight line instead of wiggly yellow curve

# Variables need to be related to be able to predict one from the other.

## Association

In [ ]:
#http://inferentialthinking.com/notebooks/hybrid.csv
hybrid = Table.read_table('hybrid.csv')
hybrid

In [ ]:
#Do you expect to see an association between acceleration and price? What kind of association? 

hybrid.scatter('acceleration', 'msrp')

In [ ]:
Image("image1.png", width=800, height=400)

In [ ]:
#Do you expect to see an association between mpg and price? What kind of association? 

hybrid.scatter('mpg', 'msrp')

In [ ]:
# This one looks less like a line, more curved.
# When observing whether there is an association, and whether that association is linear, 
# we only look at shape of points, not the units on the axes.
# To be more general, use standard units.

In [ ]:
# helper methods:

def standard_units(any_numbers):
    "Convert any array of numbers to standard units."
    return (any_numbers - np.mean(any_numbers)) / np.std(any_numbers)  

def standardize(t):
    """Return a table in which all columns of t are converted to standard units."""
    t_su = Table()
    for label in t.labels:
        t_su = t_su.with_column(label + ' (su)', standard_units(t.column(label)))
    return t_su

hybrid_su = standardize(hybrid.select('msrp', 'acceleration','mpg'))
hybrid_su

In [ ]:
hybrid_su.scatter('acceleration (su)', 'msrp (su)')

In [ ]:
hybrid_su.scatter('mpg (su)', 'msrp (su)')

In [ ]:
# Notice the shapes are the same, so we can do all our analysis in standard units, to be very general.

# One of these still looks more like a line than the other - can we make that more precise? 

In [ ]:
Image("image2.png", width=600, height=300)

In [ ]:
#try out different values of r

r_scatter(0.7)

In [ ]:
Image("image5.png", width=800, height=200)

In [ ]:
Image("image6.png", width=800, height=400)

In [ ]:
# Then compute the average of the rightmost column -> 0.617

# Positive association: below-average values of x usually go with below-average values of y
# and sign of the product is usually positive

In [ ]:
def correlation(t, label_x, label_y):
    return np.mean(standard_units(t.column(label_x))*standard_units(t.column(label_y)))

In [ ]:
hybrid.scatter('acceleration', 'msrp')
correlation(hybrid, 'acceleration', 'msrp')

In [ ]:
hybrid.scatter('mpg', 'msrp')
correlation(hybrid, 'mpg', 'msrp')

In [ ]:
# We know how to make predictions regardless of correlation (as we did with Galton's data).

# Correlation tells you something about accuracy of your predictions:
# higher magnitude (absolute value) of correlation means more accurate predictions

In [ ]:
Image("image4.png", width=600, height=300)

In [ ]:
# Important: Not correlated does not mean not related. 
# First visualize, then quantify 

## Regression Line

In [ ]:
#Back to Galton's data

#Trying to discover equation for yellow curve - would like a perfect line

heights_and_predict.scatter(0)

In [ ]:
# How strong is the linear trend?

correlation(heights_and_predict, 'MidParent', 'Child')

In [ ]:
# We take our heights and convert them to standard units:

standardize(heights).scatter(0)
plots.xlim(-4, 4)
plots.ylim(-4, 4)

In [ ]:
# When converting to standard units, only axes change, shape is same

In [ ]:
# Plot a natural line of symmetry, 45 degree angle - is this a good prediction line?

standardize(heights).scatter(0)
plots.xlim(-4, 4)
plots.ylim(-4, 4)
plots.plot([-4, 4], [-4, 4], color='r', lw=2)


In [ ]:
# green line: drawn at particular x value that I want to predict

standardize(heights).scatter(0)
plots.xlim(-4, 4)
plots.ylim(-4, 4)
plots.plot([-4, 4], [-4, 4], color='r', lw=2)
plots.plot([2.5, 2.5], [-4, 4], color='g', lw=2)

In [ ]:
#blue line goes through centers of vertical strips - it is flatter

standardize(heights).scatter(0)
plots.xlim(-4, 4)
plots.ylim(-4, 4)
plots.plot([-4, 4], [-4, 4], color='r', lw=2)
plots.plot([2.5, 2.5], [-4, 4], color='g', lw=2)

r = correlation(heights, 0, 1)
plots.plot([-4, 4], [-4*r, 4*r], color='dodgerblue', lw=2)

In [ ]:
# notice that blue and red lines both go through (0,0)

# What is slope of blue line? 

# Positive or negative? 

# More or less than one?

# blue line is called a regression line, goes through middle points

# Can you find equation of a line that goes through origin with a given slope?

In [ ]:
Image("image3.png", width=700, height=150)

In [ ]:
# Consider height of child whose parents are extremely tall

# (back to slides - graph of averages)

## Regression Line for Prediction

In [ ]:
# Question: How does the regression line compare to our original prediction?

heights_and_predict.scatter(0)

In [ ]:
r = correlation(heights_and_predict, 'MidParent', 'Child')
parent_mean = np.mean(heights.column('MidParent'))
parent_sd = np.std(heights.column('MidParent'))
child_mean = np.mean(heights.column('Child'))
child_sd = np.std(heights.column('Child'))

In [ ]:
def predict_with_r(parent):
    """Return a prediction of the height of a child 
    whose parents have a midparent height of parent, 
    using linear regression.
    """
    parent_su = (parent - parent_mean) / parent_sd
    child_su = r * parent_su
    return child_su * child_sd + child_mean

In [ ]:
# Parent -> child (below the parents' height)

predict_with_r(68)

In [ ]:
# Parent -> child (below the parents' height)

predict_with_r(74)

In [ ]:
# Parent -> child 

predict_with_r(60)

In [ ]:
#Now with all the midparent heights

heights_and_predict.with_column(
    'Prediction with r', 
    heights_and_predict.apply(predict_with_r, 'MidParent')).scatter(0)


In [ ]:
#(back to slides - slope and intercept)

## Slope and Intercept

In [ ]:
# How to work in original units?

def correlation(t, x, y):
    return np.mean(standard_units(t.column(x))*standard_units(t.column(y)))

def slope(t, x, y):
    """The slope of the regression line (original units)"""
    r = correlation(t, x, y)
    return r * np.std(t.column(y)) / np.std(t.column(x))

def intercept(t, x, y):
    """The intercept of the regression line (original units)"""
    return np.mean(t.column(y)) - slope(t, x, y) * np.mean(t.column(x))


In [ ]:
hybrid.scatter('acceleration', 'msrp', fit_line = True)

# Make predictions with the line y=ax+b

a = slope(hybrid, 'acceleration', 'msrp')
b = intercept(hybrid, 'acceleration', 'msrp')

print(a, b)

In [ ]:
# Predict the retail price of a hybrid with acceleration of 12.5

a*12.5+b